# Spark SQL Básico

Spark SQL es el módulo de Apache Spark para trabajar con datos estructurados (esquema definido). Permite ejecutar consultas SQL o usar una API estilo SQL (como .select, .where, etc.) sobre DataFrames, tanto en lenguaje SQL como en APIs de alto nivel (PySpark, Scala, etc.). En este laboratorio, nos centraremos únicamente en consultas mediante lenguaje SQL y la creación de vistas temporales.

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PySpark05").getOrCreate()

In [ ]:
data = [
    ("Alice", "HR", 3000),
    ("Bob", "IT", 4500),
    ("Cathy", "HR", 3200),
    ("David", "IT", 5000),
    ("Eve", "Finance", 4000)
]
columns = ["name", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

## Vistas temporales

Las vistas temporales (temp views) son una forma de registrar un DataFrame como una tabla SQL temporal en la sesión de Spark. Esto permite usar SQL sobre cualquier DataFrame que ya hayas cargado o transformado. Esto no guarda nada en disco, y solo vive mientras dure la sesión de Spark.

Existe una variante que no se borra al cerrar la sesión, sino que se comparte entre sesiones Spark:
```python
df.createGlobalTempView("nombre_global")
spark.sql("SELECT * FROM global_temp.nombre_global")
```

##### ¿Cuándo usar vistas temporales?
- Cuando vienes del mundo SQL y te es más natural escribir consultas.
- Cuando quieres dividir la lógica de procesamiento en etapas (puedes crear una vista para cada paso).
- Cuando compartes lógica entre scripts o notebooks y necesitas una "tabla virtual" común.
- Cuando haces prototipado rápido.

In [ ]:
# Crear una vista temporal
df.createOrReplaceTempView("employees")

# Ejecutar consulta SQL
result = spark.sql("SELECT department, AVG(salary) AS avg_salary FROM employees GROUP BY department")
result.show()

## Funciones integradas y expresiones SQL

Con Spark, podemos lanzar consultas SQL directamente como una query sin tener que traducirla a PySpark.

In [ ]:
# Filtros y orden
spark.sql("SELECT name, salary FROM employees WHERE salary > 3500 ORDER BY salary DESC").show()

# Conteo por categoría
spark.sql("SELECT department, COUNT(*) as count FROM employees GROUP BY department").show()

Spark permite aplicar funciones como: AVG, COUNT, SUM, DATEDIFF, CASE WHEN, ROUND, etc.

## Equivalente en código Python (API DataFrame)

In [ ]:
df.filter(df.salary > 3500).select("name", "salary").orderBy("salary", ascending=False).show()
df.groupBy('department').agg(f.count('*').alias('num_employees')).show()

## Ejercicio

1. Carga el archivo *../data/taxi_zone_lookup.csv* que contiene datos relacionados con zonas de recogida de taxis y crea una vista temporal
2. Contesta a las preguntas:
    - ¿Cuántas zonas hay por cada borough?
    - ¿Cuántos LocationID hay por zona de servicio?
3. Compara las mismas operaciones usando la API de DataFrame.

In [ ]:
# Cargar datos relacionados con zonas de recogida de taxis
df = spark.read.option("header", True).csv("../../data/taxi_zone_lookup.csv")

# Ver las primeras filas
df.show(5)
# Esquema del DataFrame
df.printSchema()

df.createOrReplaceTempView("taxi_zones")

In [ ]:
# ¿Cuántas zonas hay por cada borough?
spark.sql("""
    SELECT Borough, COUNT(DISTINCT Zone) AS num_zones
    FROM taxi_zones
    GROUP BY Borough
    ORDER BY num_zones DESC
""").show()

In [ ]:
# ¿Cuántos LocationID hay por zona de servicio?
spark.sql("""
    SELECT service_zone, COUNT(*) AS location_count
    FROM taxi_zones
    GROUP BY service_zone
""").show()

In [ ]:
# ¿Cuántas zonas hay por cada borough?
df.groupBy('Borough').agg(f.countDistinct('Zone').alias('zones')).orderBy('zones', ascending=False).show()

In [ ]:
# ¿Cuántos LocationID hay por zona de servicio?
df.groupBy('service_zone').agg(f.count('*').alias('count')).show()

In [ ]:
spark.stop()